In [ ]:
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
# === Load dataset ===

# Load only the CUSTOMER section of the text file
data = pd.read_fwf("RC104_25.txt", skiprows=8, header=None)
data.columns = ["CUST_NO", "XCOORD", "YCOORD", "DEMAND", "READY_TIME", "DUE_DATE", "SERVICE_TIME"]


# Load data from previous period
data_pre = pd.read_fwf("RC103_25.txt", skiprows=8, header=None)
data_pre.columns = ["CUST_NO", "XCOORD", "YCOORD", "DEMAND", "READY_TIME", "DUE_DATE", "SERVICE_TIME"]

# === Sets ===
V = list(range(25))  # 25 vehicles
C = list(range(1, len(data)))  # Customers (1 to n), excluding depot (0)
N = list(range(len(data)))  # All nodes including depot

# === Parameters ===
dc = data['DEMAND'].tolist()
dc[0] = 0  # Depot demand = 0
dc_pre = data_pre['DEMAND'].tolist()
dc_pre[0] = 0  # Depot demand = 0
coords = list(zip(data['XCOORD'], data['YCOORD']))
ai = data['READY_TIME'].tolist()
bi = data['DUE_DATE'].tolist()
si = data['SERVICE_TIME'].tolist()

# Problem parameters
cij = 30
delta = 0.17        #https://progresschamber.org/wp-content/uploads/2024/06/Chamber-of-Progress-Efficiency-and-Emissions-Impact-of-Last-Mile-Online-Delivery-in-the-US.pdf
lambda_v = 200
pi = 10
theta = 0.7
mv = 100000
cv = 100
hv = 15
ev = 0.1
cf = 0.5                                                                                        
fv = 0.1
M = 1000000
w1, w2, w3 = 1, 1, 1  # weight for objectives

dc_rev = [round(d * delta) for d in dc_pre]  # Integer reverse demand!!!!

# === Distance matrix ===
t = np.zeros((len(N), len(N)))
for i in N:
    for j in N:
        t[i][j] = math.hypot(coords[i][0] - coords[j][0], coords[i][1] - coords[j][1])


In [ ]:
# === Metaheuristic LNS + SA Implementation ===

import random
import math
import copy
import json

# === Load initial routes from output file of exact model ===
with open("routes.json", "r") as f:
    initial_routes = json.load(f)

# === Best single-objective values from exact model (used for normalization) ===
Feco_best = 18390.9967
Fenv_best = 30.4283
Fsoc_best = 411.1106

# === Evaluate a solution using normalized weighted gaps and demand satisfaction ===
def evaluate_solution(routes, unsat_delivery, unsat_return):
    Feco = Fenv = Fsoc = 0

    for route in routes:
        if not route or len(route) < 2:
            continue
        time = 0
        last = route[0]
        for idx in range(1, len(route)):
            curr = route[idx]
            dist = t[last][curr]
            Feco += dist * (cf * fv + cij)
            Fenv += ev * dist
            time += dist + (si[curr] if curr in C else 0)
            last = curr
        Feco += cv + hv * time
        Fsoc += time

    Fsat = sum(theta * unsat_delivery[i] + (1 - theta) * unsat_return[i] for i in C)
    F1 = w1 * (Feco - Feco_best) / Feco_best + \
         w2 * (Fenv - Fenv_best) / Fenv_best + \
         w3 * (Fsoc - Fsoc_best) / Fsoc_best

    return F1 + Fsat  

# === Randomly destroy a portion of customers ===
def random_destroy(routes, fraction=0.6):
    all_customers = [(v, cust) for v in range(len(routes)) for cust in routes[v] if cust != 0]
    num_remove = int(len(all_customers) * fraction)
    to_remove = random.sample(all_customers, num_remove)
    removed = []
    for v, cust in to_remove:
        if cust in routes[v]:
            routes[v].remove(cust)
            removed.append(cust)
    return removed, routes

# === Greedy repair ===
def greedy_repair(routes, removed):
    for cust in removed:
        best_cost = float('inf')
        best_pos = None
        for v in range(len(routes)):
            for i in range(len(routes[v])):
                temp_routes = copy.deepcopy(routes)
                temp_routes[v].insert(i, cust)
                cost = evaluate_solution(temp_routes, dc, dc_rev)
                if cost < best_cost:
                    best_cost = cost
                    best_pos = (v, i)
        if best_pos:
            v, i = best_pos
            routes[v].insert(i, cust)
    return routes


# === LNS + SA main loop ===
def lns_sa(initial_routes, iterations=1000, initial_temp=200, cooling_rate=0.98, destroy_fraction=0.5):
    current_routes = copy.deepcopy(initial_routes)
    best_routes = copy.deepcopy(current_routes)
    best_cost = evaluate_solution(current_routes, dc, dc_rev)
    temperature = initial_temp

    for it in range(iterations):
        removed, partial_routes = random_destroy(copy.deepcopy(current_routes), fraction=destroy_fraction)
        repaired_routes = greedy_repair(partial_routes, removed)

        new_cost = evaluate_solution(repaired_routes, dc, dc_rev)
        old_cost = evaluate_solution(current_routes, dc, dc_rev)

        delta = new_cost - old_cost
        if delta < 0 or random.random() < math.exp(-delta / temperature):
            current_routes = copy.deepcopy(repaired_routes)
            if new_cost < best_cost:
                best_cost = new_cost
                best_routes = copy.deepcopy(repaired_routes)

        temperature *= cooling_rate

        if it % 100 == 0:
            print(f"Iteration {it}: Current = {old_cost:.2f}, Best = {best_cost:.2f}")

    return best_routes, best_cost

# === Run LNS + SA ===
lns_routes, lns_cost = lns_sa(initial_routes)

print("\nFinal LNS + SA Cost:", lns_cost)
for v, route in enumerate(lns_routes):
    if route:
        print(f"Vehicle {v}: {route}")


Iteration 0: Current = 404.46, Best = 404.40
Iteration 100: Current = 404.37, Best = 404.31
Iteration 200: Current = 404.38, Best = 404.31
Iteration 300: Current = 404.36, Best = 404.31
Iteration 400: Current = 404.39, Best = 404.31
Iteration 500: Current = 404.33, Best = 404.31
Iteration 600: Current = 404.32, Best = 404.31
Iteration 700: Current = 404.32, Best = 404.31
Iteration 800: Current = 404.30, Best = 404.30
Iteration 900: Current = 404.30, Best = 404.30

Final LNS + SA Cost: 404.3043734047083
Vehicle 3: [0, 2, 6, 7, 8, 4, 5, 3, 1]
Vehicle 7: [0, 10, 11, 12, 14, 17, 16, 15, 13, 9]
Vehicle 11: [0, 24, 22, 20, 19, 18, 21, 23, 25]
